In [1]:
import numpy as np
import numpy.linalg as la
import cvxpy as cp

from functools import partial

  
import matplotlib.pyplot as plt
import labellines as ll


from shapely.geometry import Point
from descartes import PolygonPatch

from RKHS import feature

In [3]:
# The differentiable kernel function with parameters c,l not filled.
def h(r,c,l):
    return c * np.exp(-(r**2) / l**2)

def k(x1,x2,c,l):
    small_sig = 1e-10 # This is needed for numerical stability.
    return h(np.linalg.norm(x1-x2+small_sig,axis = -1),c,l)

def Gram(kernel,x):
    KA = kernel(x[:,np.newaxis,:],x)
    return KA

In [2]:
def h_inv(y,c,l):
    return np.sqrt(np.log(c/y))*l

In [4]:
kernel = partial(k,c=1,l=1)

# Set up the reference points

In [5]:
nx, ny = (10, 10)
x = np.linspace(-2, 4, nx)
y = np.linspace(-2, 4, ny)
xv, yv = np.meshgrid(x, y)

ref = np.array([xv.ravel(),yv.ravel()]).T

In [6]:
ref[33]

array([0., 0.])

In [81]:
radius = 2
Rs = np.ones(len(ref))*8
Rs[33] = radius

K = Gram(kernel,ref)

l,U =la.eig(K)

# M = np.diag(1/np.sqrt(l)).dot(U) # M.T.dot(M) = K^{-1} is satisfied.

# Another idea of choosing M, not from the eigen decomp approach though.
M = la.cholesky(la.inv(K)).T

# Define some constants

In [35]:
N = len(ref)
T = 7

c = 1
l = 1
var_0 = 0.01

step_size = 1

In [36]:
n = T

d = h(step_size,c=c,l=l)

I = np.eye(n)

# Define the decision variables that represent the feature matrix, solve for an feasible solution.

In [82]:
H = h(Rs,c=1,l=1).reshape(-1,1)

H.dot(np.ones((1,T))).shape

phi = cp.Variable((N,T))

constraints = [la.inv(M) @ phi>= H.dot(np.ones((1,T)))]
# constraints = [la.inv(M) @ phi>= 0]


prob=cp.Problem(cp.Maximize(0),constraints)

prob.solve()

phi0 = phi.value

phi_prev=phi0

# Perform the alternating solver

In [84]:
for _ in range(100):
    S = phi.T @ phi_prev

    constraints = [la.inv(M) @ phi>= H.dot(np.ones((1,T)))]

    constraints += [cp.diag(S)==c,S>>0]

    constraints += [S[i,i+1]>=d for i in range(0,n-1)]
    
    prob = cp.Problem(cp.Maximize(1/2*cp.log_det(var_0*I+ S)-10*cp.norm(phi-phi_prev)**2),constraints)
    prob.solve()
    
    print(1/2*np.log(la.det(var_0*I+S.value)),0.1*la.norm(phi_prev.T.dot(phi_prev)-S.value))
  
    phi_prev = phi.value
    
#     if prob.value<1e-7:
#         break

-1.7108865435522007 5.803454223655429
-1.7411126086345252 0.3725566062442491
-1.7207297150556842 5.707339059923449
-1.7382677725892657 0.37120741759779113
-1.7263202830372515 5.653374856522692
-1.7354971036614195 0.37045776830865074
-1.7273739170351872 5.624322084574614
-1.7322566581013081 0.3700146006214199
-1.7258197984803576 5.608407794227307
-1.728321880475486 0.36973276716179393
-1.720607388052637 5.599998431089673
-1.7228183258907812 0.36951052520476685
-1.7148715486195996 5.593589961780294
-1.7173107421907736 0.3692892308756339
-1.710195031420622 5.586688184792635
-1.7124645785318138 0.3690841666614657
-1.7068059328783842 5.580475984264929
-1.70873965175334 0.3688759686663349
-1.703583077614688 5.575349742346211
-1.705154189903053 0.36874806243807173
-1.7007081345304256 5.571190192566959
-1.7020153271201643 0.3685840780439631
-1.697858136241327 5.567773281187478
-1.6989324702896322 0.36844903387251554
-1.6948883749982873 5.565075701637894
-1.695915824258274 0.36831034791185924
-

SolverError: Solver 'SCS' failed. Try another solver, or solve with verbose=True for more information.

In [65]:
1/2*np.log(la.det(var_0*I+phi_prev.T.dot(phi_prev)))

7.605673454720092

In [70]:
phi_prev.T.dot(phi_prev)

array([[ 8.26716515,  0.51656544,  0.58929813,  0.41102509,  0.24599908,
         0.06773517,  0.11096262],
       [ 0.51656544,  8.27150651,  0.49175295,  0.5226798 ,  0.20970506,
         0.06435063,  0.06548942],
       [ 0.58929813,  0.49175295,  8.27522588,  0.63713662,  0.28467973,
         0.09090907,  0.05026665],
       [ 0.41102509,  0.5226798 ,  0.63713662,  8.26149668,  0.3259984 ,
         0.06804373,  0.06522157],
       [ 0.24599908,  0.20970506,  0.28467973,  0.3259984 ,  8.26886702,
         0.19122735, -0.08584016],
       [ 0.06773517,  0.06435063,  0.09090907,  0.06804373,  0.19122735,
         8.26897177,  0.06650896],
       [ 0.11096262,  0.06548942,  0.05026665,  0.06522157, -0.08584016,
         0.06650896,  8.27801511]])

In [67]:
1/2*np.log(la.det(var_0*I+S.value))

-0.12179673379967382

In [68]:
S.value

array([[ 1.00000153,  0.36788099,  0.17680445,  0.26734446,  0.18957795,
         0.19205686,  0.03208584],
       [-0.06404633,  1.00000219,  0.36788048,  0.31036128,  0.24018062,
         0.16065981,  0.10141061],
       [ 0.44705635,  0.26340965,  1.0000029 ,  0.44845947,  0.40465098,
         0.27209874,  0.11935936],
       [ 0.14406011,  0.13262937,  0.15749033,  0.99999979,  0.36788053,
         0.30497409, -0.18677922],
       [ 0.30682769,  0.32298222,  0.31963231,  0.09980217,  1.00000248,
         0.36787943, -0.24423703],
       [ 0.02199849,  0.02475791,  0.04398186, -0.10249021,  0.03354682,
         1.00000572,  0.36788252],
       [-0.00994607,  0.02088042,  0.02799893,  0.16638997, -0.04698762,
        -0.09156243,  1.00000239]])

# Applying the alternating solver on the log-det is quite unstable and slow, much less efficient compared with the norm problem